In [1]:
### automatically refresh the buffer

%load_ext autoreload
%autoreload 2

### solve the auto-complete issue

%config Completer.use_jedi = False

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)

### lvl 2 setups (systerm)

import os
import numpy as np
import pandas as pd
import xarray as xr

import matplotlib as mpl
import cartopy.crs as ccrs
import cartopy.feature as cfeature

import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import warnings
warnings.filterwarnings('ignore')
from pylab import *
from matplotlib.colors import ListedColormap,LinearSegmentedColormap
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.patches import Wedge, Circle
import geopandas as gpd
from shapely.geometry import Point
from datetime import datetime
import glob
import datetime

In [2]:
gdf = gpd.read_file('/N/project/Zli_lab/Data/Other/tl_2019_us_state/tl_2019_us_state.shp')
input_folder = '/N/project/Zli_lab/Data/Observations/NCAR/prec_acc_files/'

start_year = 2015
end_year = start_year+3
for year in range(start_year, end_year):  # 1989不包含

    months = range(10, 13) if year == start_year else range(1, 10) if year == (end_year - 1) else range(1, 13)
    # 遍历月份
    for month in months:
        print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()))
        # 获取当前月份的天数
        if month in [1, 3, 5, 7, 8, 10, 12]:
            num_days = 31
        elif month in [4, 6, 9, 11]:
            num_days = 30
        elif month == 2:
            # 考虑闰年
            if (year % 4 == 0 and year % 100 != 0) or (year % 400 == 0):
                num_days = 29  # 闰年
            else:
                num_days = 28  # 平年

        # 遍历每个月的天数
        for day in range(1, num_days + 1):
            
            month_str = f"{month:02}"
            day_str = f"{day:02}"
            input_file = f'PREC_ACC_NC.wrf2d_d01_{year}-{month_str}-{day_str}_*.nc'
            ds = xr.open_mfdataset(input_folder + input_file)
            ds_era = xr.Dataset({'p': (['time', 'latitude', 'longitude'], ds.PREC_ACC_NC.values)},
                                coords={'longitude': (['longitude'], ds.XLONG.values[500]),
                                        'latitude': (['latitude'], ds.XLAT.values[:,500]),
                                        'time': ('time', ds.Time.values)})
            ds_era_lon, ds_era_lat = np.meshgrid(ds_era.longitude.values, ds_era.latitude.values, indexing='xy')
            # 转换为 xarray DataArray，确保其维度与 ds_era_clipped 对齐
            ds_era_lon_da = xr.DataArray(ds_era_lon, dims=("latitude", "longitude"), coords={"latitude": ds_era.latitude, "longitude": ds_era.longitude})
            ds_era_lat_da = xr.DataArray(ds_era_lat, dims=("latitude", "longitude"), coords={"latitude": ds_era.latitude, "longitude": ds_era.longitude})

            # 使用 assign_coords 将二维坐标添加到 ds_era_clipped
            ds_era_clipped = ds_era.assign_coords(lon_2d=ds_era_lon_da, lat_2d=ds_era_lat_da)

            lon = ds_era_clipped['lon_2d'].values
            lat = ds_era_clipped['lat_2d'].values
            grid = gpd.GeoDataFrame(
                geometry=gpd.points_from_xy(lon.flatten(), lat.flatten()),
                index=np.arange(lon.size)
            )
            grid.set_crs(gdf.crs, inplace=True)
            grid_s = gpd.sjoin(grid, gdf, how='inner', predicate='within')

            mask = np.full(ds_era_clipped['p'].shape[1:], False) 
            for index in grid_s.index:
                row, col = np.unravel_index(index, mask.shape)  # 获取行列索引
                mask[row, col] = True
            mask_da = xr.DataArray(mask, dims=ds_era_clipped['p'].dims[1:], coords={'latitude': ds_era_clipped['p'].coords['latitude'], 'longitude': ds_era_clipped['p'].coords['longitude']})
            ds_sss = ds_era_clipped.where(mask_da, drop=True)
            ds_sss = ds_sss.drop_vars(['lon_2d', 'lat_2d'])


            original_times = ds_sss.time.values 
            
            lon_ranges = [(-np.inf, -112.5), (-112.5, -97.5), (-97.5, -82.5), (-82.5, np.inf)]
            utc_offsets = [-8, -7, -6, -5]
            print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()))
            for (lon_min, lon_max), offset in zip(lon_ranges, utc_offsets):
                mask = (ds_sss.longitude >= lon_min) & (ds_sss.longitude < lon_max)
                ds_lon_subset = ds_sss.where(mask, drop=True)
                if ds_lon_subset.latitude.size > 0 and ds_lon_subset.longitude.size > 0:
            
                    adjusted_times = original_times + np.timedelta64(offset, 'h')  # 保持24个时间点
            
                    ds_lon_subset = ds_lon_subset.assign_coords(time=adjusted_times)
            
                    lat_min = ds_lon_subset.latitude.min().values
                    lat_max = ds_lon_subset.latitude.max().values
            
                    lat_splits = np.linspace(lat_min, lat_max, 10)  # 10个值分9段
                    
                    for i in range(len(lat_splits) - 1):
                        lat_min_split = lat_splits[i]
                        lat_max_split = lat_splits[i + 1]
                        lat_mask = (ds_lon_subset.latitude >= lat_min_split) & (ds_lon_subset.latitude < lat_max_split)
                        ds_lat_subset = ds_lon_subset.where(lat_mask, drop=True)
                        
                        output_folder = '/N/project/Zli_lab/gongg/CONUS404_data/LST/re_UTC/U' + str(offset)+str(i) + '/'
                        output_file = f'PREC_ACC_NC.wrf2d_d01_{year}-{month_str}-{day_str}.nc'
                        output_path = os.path.join(output_folder, output_file)
                        os.makedirs(output_folder, exist_ok=True)
                        ds_lat_subset.to_netcdf(output_folder + output_file)

2025-02-17 01:39:06
2025-02-17 01:39:37
2025-02-17 01:40:05
2025-02-17 01:40:31
2025-02-17 01:41:01
2025-02-17 01:41:26
2025-02-17 01:41:55
2025-02-17 01:42:26
2025-02-17 01:42:48
2025-02-17 01:43:14
2025-02-17 01:43:39
2025-02-17 01:44:06
2025-02-17 01:44:37
2025-02-17 01:45:05
2025-02-17 01:45:36
2025-02-17 01:46:00
2025-02-17 01:46:31
2025-02-17 01:46:59
2025-02-17 01:47:24
2025-02-17 01:47:47
2025-02-17 01:48:13
2025-02-17 01:48:37
2025-02-17 01:49:05
2025-02-17 01:49:31
2025-02-17 01:49:59
2025-02-17 01:50:26
2025-02-17 01:50:51
2025-02-17 01:51:19
2025-02-17 01:51:43
2025-02-17 01:52:08
2025-02-17 01:52:30
2025-02-17 01:52:53
2025-02-17 01:52:58
2025-02-17 01:53:17
2025-02-17 01:53:38
2025-02-17 01:54:02
2025-02-17 01:54:29
2025-02-17 01:54:55
2025-02-17 01:55:26
2025-02-17 01:55:56
2025-02-17 01:56:22
2025-02-17 01:56:50
2025-02-17 01:57:17
2025-02-17 01:57:44
2025-02-17 01:58:15
2025-02-17 01:58:44
2025-02-17 01:59:13
2025-02-17 01:59:41
2025-02-17 02:00:08
2025-02-17 02:00:36
